## 🚀 Ascent AI: Intelligent Academic Concierge 🎓

**Ascent AI** is a **multi-agent academic and university counseling system** designed to transform a student's confusion into a **clear, actionable plan**.

### 🌟 Core Function

Ascent AI functions as a **personalized, long-term academic and university counselor** for high school students. Its primary goal is to guide students to a defined outcome:

* **From :** "I'm uncertain about my future and next steps."
* **To :** "I know my track, target schools, and prioritized steps for this year."

### ⚙️ Technological Foundation

The system is built for **reliability and advanced reasoning**, showcasing an agent architecture:

* Built using **Google's Agent Development Kit (ADK)** for robust agent orchestration.
* Powered by **Gemini's advanced reasoning capabilities** to handle complex mapping and planning tasks.
* Features a **Sequential Multi-Agent Pipeline** (including Profile, Mapper, University, and Mentor Agents).
* Utilizes **Sessions & Memory** for persistent guidance across long-running student engagement.
* Custom Tool: `UniversitySearchTool` for structured university filtering

That's a very clear and well-structured project summary.

Here is the rephrased version, formatted for maximum impact and readability, emphasizing the value proposition and the solution's intelligence.

---

## ✨ Project Summary: Solving High School Decision Paralysis with AI 🎓

### The Challenge: Decision Paralysis in High School

The academic journey for high school students is often marked by **decision paralysis** due to critical gaps in guidance:

* **Lack of Structure:** Students are overwhelmed by options and lack a clear, **structured process** for career and academic planning.
* **Limited Access:** High **student-to-counselor ratios** prevent the necessary sustained, personalized attention.
* **Actionable Gap:** Students typically struggle to connect their abstract **interests to realistic, actionable academic tracks** and timed application steps.

---

### The Solution: Ascent AI – Your Intelligent Academic Concierge

**Ascent AI** is a **Multi-Agent Academic Planning System** designed to deliver personalized, 24/7 guidance. The system moves students from confusion to **confidence and clarity** through a structured, multi-step pipeline:

1.  **🔍 Structured Profiling & Persistence:** The system **Profiles** the student by extracting interests, grades, and constraints into a standardized format, ensuring this data is **stored persistently** for long-term use.
2.  **🗺️ Interest-to-Major Mapping:** It intelligently **Maps** the student's unique profile and strengths to 2-3 **plausible academic tracks/majors**, eliminating subjective guesswork.
3.  **🎯 Constrained University Shortlisting:** It **Shortlists universities** into defined **Reach, Target, and Safe tiers**, rigorously filtering candidates based on realistic constraints like **budget, grades, and location**.
4.  **🗓️ Actionable Roadmap Generation:** It **Generates a comprehensive, grade-specific action plan** spanning 12-24 months, detailing essential steps (exams, projects, applications).
5.  **🔄 Continuous Guidance (Mentor Agent):** The dedicated **Mentor Agent** follows up over time, intelligently revisiting the original plan, absorbing student progress updates, and **updating the roadmap** to maintain momentum and relevance.

In [31]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print("🔑 Authentication Error: Please add 'GOOGLE_API_KEY' to your Kaggle secrets.")
    raise e

✅ Gemini API key setup complete.


In [32]:
from typing import Any, Dict, List
import json

from google.genai import types

from google.adk.agents import Agent, LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import AgentTool
from google.adk.tools.tool_context import ToolContext

print("✅ ADK components imported successfully.")

# Retry configuration
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Session service (conceptual)
# Used to manage state on behalf of a client, stores session id and imp data
session_service = InMemorySessionService()
print("✅ Session service (conceptual) created.")


def pretty_print_json(data: Any):
    print(json.dumps(data, indent=2, ensure_ascii=False))

print("✅ Helper pretty printer ready.")

✅ ADK components imported successfully.
✅ Session service (conceptual) created.
✅ Helper pretty printer ready.


### Customizing the University Database (Crucial Step!) 🌍

The **UniversitySearchTool** currently operates on a mock database named **`UNIVERSITY_DB`**.

**To personalize the results of this agent for any student/school, you MUST customize this database.**

* **Locate Code Below** (the one defining `UNIVERSITY_DB`).
* **Edit the `UNIVERSITY_DB` list** by adding, removing, or modifying the dictionaries within it.
* **Ensure every university entry contains:** `name`, `country`, `tuition_band` (`low`/`medium`/`high`), and boolean flags for major support (`has_cs`, `has_ds`, etc.).

By customizing this list, your agent's suggestions will instantly reflect the specific schools relevant to your use case (e.g., your local high school's historical target list).

---


## ⚠️ **Crucial Configuration Alert: Personalizing the University Database** 🌍

The integrity and relevance of **Ascent AI's** recommendations depend entirely on the content of the mock database, **`UNIVERSITY_DB`**, currently used by the **`UniversitySearchTool`**.

### **Action Required: Database Customization**

To ensure your agent provides **accurate, personalized, and actionable** guidance that reflects your specific use case (e.g., your school's historical target list), you MUST customize this database now.

### **📝 Implementation Checklist**

1.  **Locate:** Find the code section that defines the `UNIVERSITY_DB` list.
2.  **Edit:** Modify the existing list by **adding, removing, or updating** the university entries (dictionaries).
3.  **Mandatory Schema:** Every single university entry **must** adhere strictly to the following data contract for filtering to work:

    * `name`: (string)
    * `country`: (string)
    * `tuition_band`: (string, must be one of: **"low"** | **"medium"** | **"high"**)
    * `has_cs`: (boolean)
    * `has_ds`: (boolean)
    * *... include all relevant major flags for your agents.*

### **Value Proposition**

By customizing this list, you instantly tailor the agent's suggestions to the specific schools relevant to your students, making the final recommendations realistic and highly valuable.


In [34]:
# === UniversitySearchTool: custom tool used by the University Agent ===

UNIVERSITY_DB: List[Dict[str, Any]] = [
    {
        "name": "Delft University of Technology (TU Delft)",
        "country": "Netherlands",
        "tuition_band": "high",
        "has_cs": True,
        "has_ds": True,
        "has_business": False,
        "has_psychology": False,
        "notes": "Top-ranked Engineering/Tech university in NL.",
    },
    {
        "name": "University of Amsterdam (UvA)",
        "country": "Netherlands",
        "tuition_band": "medium",
        "has_cs": True,
        "has_ds": True,
        "has_business": True,
        "has_psychology": True,
        "notes": "Broad programs, strong for Business, CS, and Psychology.",
    },
    {
        "name": "Vrije Universiteit Amsterdam (VU)",
        "country": "Netherlands",
        "tuition_band": "high", # Non-EEA Bachelor's institutional fees often high (e.g., $18k-$22k)
        "has_cs": True,
        "has_ds": True,
        "has_business": True,
        "has_psychology": True,
        "notes": "Strong in AI, Computer Science, and Medical Sciences.",
    },
    {
        "name": "Eindhoven University of Technology (TU/e)",
        "country": "Netherlands",
        "tuition_band": "high", # Non-EEA Bachelor's institutional fees often high (e.g., ~$18.6k)
        "has_cs": True,
        "has_ds": True,
        "has_business": False,
        "has_psychology": False,
        "notes": "Top-tier technical university, excellent for engineering.",
    },
    {
        "name": "Maastricht University (UM)",
        "country": "Netherlands",
        "tuition_band": "medium", # Non-EEA Bachelor's institutional fees often lower (e.g., ~$10k-14k depending on program)
        "has_cs": True,
        "has_ds": True,
        "has_business": True,
        "has_psychology": True,
        "notes": "Known for Problem-Based Learning (PBL) and International Business.",
    },
    {
        "name": "University of Groningen (RUG)",
        "country": "Netherlands",
        "tuition_band": "medium", # Non-EEA Bachelor's institutional fees often lower (e.g., ~$13.5k-$19.8k depending on faculty)
        "has_cs": True,
        "has_ds": True,
        "has_business": True,
        "has_psychology": True,
        "notes": "Comprehensive university with diverse English-taught programs.",
    },
    {
        "name": "Erasmus University Rotterdam",
        "country": "Netherlands",
        "tuition_band": "medium",
        "has_cs": False,
        "has_ds": True,
        "has_business": True,
        "has_psychology": False,
        "notes": "Best known for Finance and Management (Rotterdam School of Management).",
    },
    {
        "name": "Carnegie Mellon University",
        "country": "USA",
        "tuition_band": "high",
        "has_cs": True,
        "has_ds": True,
        "has_business": False,
        "has_psychology": False,
        "notes": "Strong CS/AI program",
    },
    {
        "name": "University of Toronto",
        "country": "Canada",
        "tuition_band": "high",
        "has_cs": True,
        "has_ds": True,
        "has_business": True,
        "has_psychology": True,
        "notes": "Top global university",
    },
    {
        "name": "University of Waterloo",
        "country": "Canada",
        "tuition_band": "high",
        "has_cs": True,
        "has_ds": True,
        "has_business": False,
        "has_psychology": False,
        "notes": "Excellent for CS/Engineering",
    },
    {
        "name": "National University of Singapore",
        "country": "Singapore",
        "tuition_band": "high",
        "has_cs": True,
        "has_ds": True,
        "has_business": True,
        "has_psychology": False,
        "notes": "Top Asian tech school",
    },
    {
        "name": "University of Twente (UT)",
        "country": "Netherlands",
        "tuition_band": "medium",
        "has_cs": True,
        "has_ds": False,
        "has_business": False,
        "has_psychology": False,
        "notes": "Strong focus on Engineering and Applied Sciences.",
    },
    {
        "name": "Arizona State University",
        "country": "USA",
        "tuition_band": "medium",
        "has_cs": True,
        "has_ds": True,
        "has_business": True,
        "has_psychology": False,
        "notes": "Good CS and Data Science options",
    },
    {
        "name": "BITS Pilani",
        "country": "India",
        "tuition_band": "medium",
        "has_cs": True,
        "has_ds": True,
        "has_business": False,
        "has_psychology": False,
        "notes": "Strong for CS and engineering in India",
    },
    {
        "name": "IIT Bombay",
        "country": "India",
        "tuition_band": "medium",
        "has_cs": True,
        "has_ds": True,
        "has_business": False,
        "has_psychology": False,
        "notes": "Premier Indian tech institute",
    },
    {
        "name": "UT Dallas",
        "country": "USA",
        "tuition_band": "medium",
        "has_cs": True,
        "has_ds": True,
        "has_business": True,
        "has_psychology": False,
        "notes": "Popular for CS/data with moderate tuition",
    },
    {
        "name": "Michigan State University",
        "country": "USA",
        "tuition_band": "low",
        "has_cs": True,
        "has_ds": False,
        "has_business": True,
        "has_psychology": True,
        "notes": "Affordable local option",
    },
    {
        "name": "Azimji Premji University",
        "country": "India",
        "tuition_band": "low",
        "has_cs": False,
        "has_ds": False,
        "has_business": True,
        "has_psychology": True,
        "notes": "Business and psychology focused",
    },
]


def UniversitySearchTool(profile: Dict[str, Any], main_track: str) -> dict:
    """
    Custom Tool: Filters universities based on profile constraints and chosen track.

    This function is used by the University Agent to separate universities into:
      - reach
      - target
      - safe

    Args:
        profile: structured student profile (including constraints)
        main_track: the main recommended track/major string

    Returns:
        dict: {
          "status": "success",
          "main_track": "...",
          "reach": [...],
          "target": [...],
          "safe": [...]
        }
    """

    
    
    constraints = profile.get("constraints", {})
    countries = constraints.get("countries")
    budget = constraints.get("budget_band")
    print(budget)

    main_track = main_track.lower()

    MAJOR_CATEGORIES = [
    ("cs", ["computer", "cs", "ai", "software"]),
    ("ds", ["data", "analytics"]),
    ("business", ["business", "management", "entrepreneur"])]

    main_track_flags = {}
    final_uni_list = []

    []

    for flag, keywords in MAJOR_CATEGORIES:
        if any(keyword in main_track for keyword in keywords):
            main_track_flags[flag] = True
    
    for uni in UNIVERSITY_DB:
        if budget == uni['tuition_band'] and countries in uni['country']:
            for key, value in main_track_flags.items():
                if uni.get(f'has_{key}') and uni[f'has_{key}']:
                    final_uni_list.append(uni)
                    break

    reach = final_uni_list[:2]
    target = final_uni_list[2:5]
    safe = final_uni_list[5:]

    result = {
        "status": "success",
        "main_track": main_track,
        "reach": reach,
        "target": target,
        "safe": safe
    }
print("✅ UniversitySearchTool defined.")

✅ UniversitySearchTool defined.


In [35]:
# === Memory Bank: Long-term student profile + plan storage ===

STUDENT_MEMORY: Dict[str, Dict[str, Any]] = {}
PLAN_MEMORY: Dict[str, Dict[str, Any]] = {}


def save_profile_tool(student_id: str, profile: Dict[str, Any]) -> dict:
    """
    Tool: Save or update a student's profile in the memory bank.
    """
    STUDENT_MEMORY[student_id] = profile
    return {"status": "success", "student_id": student_id}


def get_profile_tool(student_id: str) -> dict:
    """
    Tool: Retrieve a stored student profile from the memory bank.
    """
    profile = STUDENT_MEMORY.get(student_id)
    if profile is None:
        return {"status": "error", "error_message": f"No profile found for {student_id}"}
    return {"status": "success", "profile": profile}


def save_plan_tool(student_id: str, plan: Dict[str, Any]) -> dict:
    """
    Tool: Save or update a student's action plan in the memory bank.
    """
    PLAN_MEMORY[student_id] = plan
    return {"status": "success", "student_id": student_id}


def get_plan_tool(student_id: str) -> dict:
    """
    Tool: Retrieve a stored student action plan from the memory bank.
    """
    plan = PLAN_MEMORY.get(student_id)
    if plan is None:
        return {"status": "error", "error_message": f"No plan found for {student_id}"}
    return {"status": "success", "plan": plan}


print("✅ Memory Bank tools defined (profiles + plans).")

✅ Memory Bank tools defined (profiles + plans).


## 2. Agent Architecture & Sequential Data Flow

Ascent AI is implemented as a **Sequential Multi-Agent System** where data moves strictly from one agent to the next. 


1.  **Profile Agent:** Gathers and structures student data into a **Student Profile JSON**.
    * **Action:** Calls `save_profile_tool()` for persistence.
2.  **Mapper Agent:** Receives the profile.
    * **Action:** Uses Gemini to select the **`main_track`** (major).
3.  **University Agent:** Receives the profile and track.
    * **Action:** Uses `UniversitySearchTool` to return a **Reach/Target/Safe university shortlist**.
4.  **Action Plan Agent (Implicit):** Uses all prior data.
    * **Action:** **Generates and saves the plan (JSON)** and produces the **final human-readable Markdown report**.
5.  **Mentor Agent (Loop Agent):** Supports long-running guidance.
    * **Action:** Uses `get_profile_tool()` and `get_plan_tool()` to recall history, accepts progress, and produces an **updated mentoring message in clear Markdown**.

In [37]:
# === 1) Profile Agent ===

profile_agent = LlmAgent(
    name="ProfileAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction = """
    You are the Profile Agent. Your primary function is data standardization and persistence.
    
    You will receive a message containing the unique 'STUDENT_ID' and a 'STUDENT_DESCRIPTION' (free-form text).
    
    Your execution MUST follow these three mandatory steps precisely:
    
    STEP 1: Data Extraction and Validation
    Analyze the provided `STUDENT_DESCRIPTION`. Extract all information and strictly map it to the following JSON schema.
    
    Data Rules:
    * **student\_id:** Must be extracted directly from the message and included in the profile.
    * **name:** If the name is explicitly mentioned, use it. If not, default to the string "Student".
    * **grade & board:** Extract the student's current grade level (as an **integer**) and the academic board (e.g., "CBSE", "ICSE", "IB", "State").
    * **interests & strengths:** Summarize these into concise lists of short strings (e.g., ["coding", "astronomy"]).
    * **constraints:**
        * **countries:** List all countries mentioned as preferences.
        * **budget\_band:** Categorize any budget information into exactly one of the three strings: `"low"`, `"medium"`, or `"high"`.
    * **scores:** Map subjects to numerical scores (e.g., {"Math": 90}). If no scores are mentioned, use an empty map: `{}`.
    
    STEP 2: Persistence (Tool Call)
    Call the 'save_profile_tool' once using a tool function call.
    
    **Arguments:** Pass the extracted `student_id` and the complete profile JSON object as the `profile` argument.
    The tool call is mandatory, regardless of data completeness.
    
    STEP 3: Final Output Generation
    Ignore the response from the tool call. Your final output MUST be ONLY the structured JSON profile.
    
    * Do NOT include any Markdown formatting (like ```json), headings, or extra conversational text.
    * The output structure MUST be wrapped under the key "profile".
    
    Final Output Structure to be returned:
    {
      "profile": {
        "student_id": "...",
        "name": "...",
        "grade": ...,
        "board": "...",
        "interests": [...],
        "strengths": [...],
        "constraints": {
          "countries": [...],
          "budget_band": "low|medium|high"
        },
        "scores": {
          "...": ...
        }
      }
    }
  
    """,
    tools=[save_profile_tool],
)

print("✅ ProfileAgent created.")

✅ ProfileAgent created.


In [38]:
# === 2) Mapper Agent ===

mapper_agent = LlmAgent(
    name="MapperAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are the Mapper Agent for Ascent AI.

You must perform two phases strictly:

## PHASE 1: Track Mapping
1. Propose 2–3 academic/career tracks or majors based on the profile.
2. Choose ONE main_track that is most aligned.

## PHASE 2: Action Plan Synthesis & Saving (CRITICAL)
3. Using the chosen main_track and the student's grade/constraints, generate the Plan JSON (summary, short_term_tasks, medium_term_tasks, long_term_tasks).
4. CRITICAL: You MUST call save_plan_tool(student_id, plan_json) with the structured plan JSON you generated.

Return ONLY this JSON: { "tracks": [ ... ], "main_track": "string" }
""",
    # Mapper Agent now needs the Plan saving tool
    tools=[save_plan_tool], 
)
print("✅ MapperAgent created.")

✅ MapperAgent created.


In [39]:
# === 3) University Agent ===
university_agent = LlmAgent(
    name="UniversityAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are the University Agent and **FINAL REPORTER** for Ascent AI.

You must follow these steps:

1. Retrieve the saved plan data using get_plan_tool(student_id).
2. Call UniversitySearchTool(profile, main_track) as a tool to get the shortlist.
3. Generate a single, comprehensive, human-readable **Markdown report** that includes:
   - All data captured from the Profile Agent.
   - The Main Track chosen by the Mapper Agent.
   - The full Reach/Target/Safe University Shortlist returned by the tool.
   - The Action Plan Timeline retrieved in Step 1.
   
**IMPORTANT OUTPUT RULE:**
- **DO NOT** return any JSON or code fences.
- The final response MUST be a single, **human-readable Markdown report** following the structure you defined.
""",
    # Tools: UniversitySearchTool and the memory retrieval tool
    tools=[UniversitySearchTool, get_plan_tool], 
)

print("✅ UniversityAgent created.")

✅ UniversityAgent created.


In [40]:
# === 5) Mentor Agent (Loop Agent) ===

mentor_agent = LlmAgent(
    name="MentorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are the Mentor Agent for Ascent AI. You support long-running, continuous guidance for the student.

You will receive:
- A student_id string
- A progress_update string (student describing what they have done since the last plan)

Your steps:
1. Use get_profile_tool(student_id) and get_plan_tool(student_id) to retrieve the original profile and plan.
2. Compare the student's progress_update with the existing timeline.
3. Generate a single, encouraging, and actionable **Markdown message** for the student.

**IMPORTANT OUTPUT RULE:**
- **DO NOT** return any JSON or code fences.
- The final response MUST be a single, **human-readable Markdown message**.

**Message Structure:**
# ⭐ Progress Check-in for [Student Name]
---
## Status & Feedback
* **Great Job:** Summarize the student's accomplishments (e.g., Python course, coding club) and relate them to their original track.
* **Next Focus:** Gently remind them of an overdue or crucial next task (e.g., SAT Prep) from the Medium-Term tasks.

## 💡 Updated Suggestions (Next 30-90 Days)
Based on your progress, focus on:
* [Refined Step 1 for the next 30 days]
* [Refined Step 2 for the next 30 days]
* [Refined Step 3 for the next 30 days]
""",
    tools=[get_profile_tool, get_plan_tool],
)

print("✅ MentorAgent revised for clear Markdown guidance.")

✅ MentorAgent revised for clear Markdown guidance.


In [41]:
# === Root Sequential Multi-Agent System ===

root_agent = SequentialAgent(
    name="AscentPipeline",
    sub_agents=[
        profile_agent,
        mapper_agent,
        university_agent,
        #action_plan_agent,
    ],
)

print("✅ Root SequentialAgent (AscentPipeline) created.")

# In this ADK version, InMemoryRunner only takes the agent
runner = InMemoryRunner(root_agent)
print("✅ InMemoryRunner created for AscentPipeline.")

# Separate runner for the MentorAgent loop
mentor_runner = InMemoryRunner(mentor_agent)
print("✅ InMemoryRunner created for MentorAgent.")


# ============= UNIVERSAL DEBUG PRINTER (Unchanged) ===============

def debug_print_events(events):
    print("\n==================== AGENT TURNS ====================\n")

    for turn in events:
        who = getattr(turn, "source", "Unknown")
        print(f"{who} >\n")

        text_found = False

        # 1) content.text
        if (
            hasattr(turn, "content")
            and turn.content is not None
            and hasattr(turn.content, "text")
            and turn.content.text
        ):
            print(turn.content.text)
            text_found = True

        # 2) content.parts[].text
        if (
            hasattr(turn, "content")
            and turn.content is not None
            and hasattr(turn.content, "parts")
            and turn.content.parts is not None
        ):
            for p in turn.content.parts:
                if hasattr(p, "text") and p.text:
                    print(p.text)
                    text_found = True

        # 3) delta.text
        if hasattr(turn, "delta") and turn.delta is not None:
            if hasattr(turn.delta, "text") and turn.delta.text:
                print(turn.delta.text)
                text_found = True

        # 4) message str
        if hasattr(turn, "message") and isinstance(turn.message, str):
            print(turn.message)
            text_found = True

        if not text_found:
            print("[No text output]")

        print("\n------------------------------\n")

✅ Root SequentialAgent (ClarityPipeline) created.
✅ InMemoryRunner created for ClarityPipeline.
✅ InMemoryRunner created for MentorAgent.


## 3. Running the Ascent AI Demo

This section executes the full sequential pipeline (`Profile` -> `Mapper` -> `University` -> `Plan`).

### 3.1 Demo Instruction: Update Your Initial Prompt

Before running the cell below, **customize the `demo_description` variable** with your own student profile. This is the free-form text input that the **Profile Agent** uses for the entire plan.

**To get the best results, you must include:**
* **Personal Info:** `My name is [Name]. I'm in Grade [Grade] [Board].`
* **Interests:** `I love [Interest 1], [Interest 2], and I'm good at [Strength].`
* **Constraints:** `I can afford [budget: low/medium/high] tuition, and I’d like to study in [Country 1] or [Country 2].`
* **Scores:** List your high-scoring subjects and grades.

---

In [45]:
# ============================================================
# DEMO: FULL MULTI-AGENT PIPELINE RUN (WITH PARSED FINAL JSON)
# ============================================================

import asyncio # Ensure asyncio is imported here

demo_description = """
My name is Dev. I recently completed a **Bachelor of Technology (B.Tech) in Computer Science** from an institute in India.
I love Artificial Intelligence (AI), deep learning, data science, and I'm looking for a specialization in Machine Learning. 
I enjoy reading research papers and competing in coding challenges.
My family can afford medium to high tuition, and I'm focused on pursuing a **Master of Science (M.Sc.) in AI in the Netherland* or Germany.
My scores: Overall GPA equivalent to 8.5/10.0 or 3.6/4.0, with top scores in Algorithms and Linear Algebra. I also have one year of experience as a Data Analyst.
"""

prompt = f"""
STUDENT_ID: student_001

STUDENT_DESCRIPTION:
{demo_description}
"""

# Reset memory for a clean run
if "student_001" in STUDENT_MEMORY:
    del STUDENT_MEMORY["student_001"]
if "student_001" in PLAN_MEMORY:
    del PLAN_MEMORY["student_001"]
print("✅ Memory reset for student_001.")


# Run through full multi-agent pipeline with debug info
# The runner handles all agent calls, tool calls, and sequential execution.
response = await runner.run_debug(prompt)


# ---- 1. FILTERED AGENT TURNS (Show the final clear Markdown output) ----
print("\n==================== AGENT TURNS (FILTERED OUTPUT) ====================\n")
for i, turn in enumerate(response):
    who = getattr(turn, "source", f"Turn {i}")
    
    text = None
    if hasattr(turn, "content") and turn.content is not None:
        if hasattr(turn.content, "text") and turn.content.text:
            text = turn.content.text
        elif hasattr(turn.content, "parts") and turn.content.parts is not None:
            # Join all text parts from the turn
            texts = [p.text for p in turn.content.parts if hasattr(p, "text") and p.text]
            if texts:
                text = ("\n".join(texts))
    
    # CRITICAL FILTER: Only print the final agent's Markdown output
    if text and who == "ActionPlanAgent":
        print(f"--- OUTPUT FROM: {who} ---")
        print(text.strip())
        print("\n------------------------------\n")


# ---- 2. CONFIRMATION OF INTERNAL JSON SAVE (Addressing the error) ----
print("\n===== CONFIRMATION OF INTERNAL JSON SAVE (Plan Memory Check) =====\n")

if PLAN_MEMORY.get("student_001"):
    print("✅ Plan successfully found in PLAN_MEMORY.")
    print("This confirms the ActionPlanAgent successfully called the save_plan_tool.")
    pretty_print_json(PLAN_MEMORY["student_001"])
else:
    print("⚠️ Plan NOT found in PLAN_MEMORY.")
    print("This indicates the ActionPlanAgent either did not call save_plan_tool or the tool call failed.")
    # Show the last response content to help debug the failure reason
    last_turn = response[-1]
    print("\n--- Last Agent Response Content (Check for missing tool call) ---")
    print(last_turn)

✅ Memory reset for student_001.

 ### Continue session: debug_session_id

User > 
STUDENT_ID: student_001

STUDENT_DESCRIPTION:

My name is Dev. I recently completed a **Bachelor of Technology (B.Tech) in Computer Science** from an institute in India.
I love Artificial Intelligence (AI), deep learning, data science, and I'm looking for a specialization in Machine Learning. 
I enjoy reading research papers and competing in coding challenges.
My family can afford medium to high tuition, and I'm focused on pursuing a **Master of Science (M.Sc.) in AI in the Netherland* or Germany.
My scores: Overall GPA equivalent to 8.5/10.0 or 3.6/4.0, with top scores in Algorithms and Linear Algebra. I also have one year of experience as a Data Analyst.


ProfileAgent > Based on your profile and your interest in Machine Learning Engineering, here are some Master's programs in AI/ML in the Netherlands and Germany that you might consider:

**Netherlands:**

*   **Delft University of Technology (TU Delft)

MapperAgent > PHASE 1: Track Mapping
1. Proposed Tracks:
    * Machine Learning Engineering
    * AI Research Scientist
    * Data Science with ML Specialization

2. Main Track: Machine Learning Engineering

PHASE 2: Action Plan Synthesis & Saving
3. Plan JSON:
```json
{
    "summary": "Pursue a Master of Science in AI/ML in the Netherlands or Germany, specializing in Machine Learning Engineering. Leverage a strong background in Computer Science, AI, deep learning, data science, and excellent scores in Algorithms and Linear Algebra. The goal is to secure a role as a Machine Learning Engineer, with a medium to high tuition budget.",
    "short_term_tasks": [
        "Thoroughly research the specific curricula and admission requirements for AI/ML Master's programs at TU Delft, University of Amsterdam, TU/e, TUM, RWTH Aachen, and Heidelberg University.",
        "Prepare and submit all application documents, including transcripts, CV, letters of recommendation, and English proficiency sco

### 3.2 Progress Update & Mentor Agent Guidance

This section demonstrates the **Loop Agent** concept. The **Mentor Agent** retrieves the saved plan and adjusts guidance based on a new user update.

### Demo Instruction: Update the Progress Prompt

Before running the cell below, **customize the `progress_update` variable** to simulate actual student progress or shifting interests.

---

In [46]:
# ============================================================
# PROGRESS UPDATE: MENTOR AGENT LOOP RUN (WITH PARSED OUTPUT)
# ============================================================

import asyncio

progress_update = """
Hi, this is Dev again. Since the last plan:
- I built a small ML project and uploaded it on GitHub. 
- I am also doing an internship at an AI startup
What should I focus on next?
"""

mentor_prompt = json.dumps(
    {
        "student_id": "student_001",
        "progress_update": progress_update,
    },
    indent=2,
)

# Use the separate runner for the Mentor Agent
mentor_runner = InMemoryRunner(mentor_agent)

# Run the Mentor Agent
mentor_response = await mentor_runner.run_debug(mentor_prompt)

# === CRITICAL FIX: AGGRESSIVE READ LOOP ===
# We will explicitly pause AND aggressively re-read the response object 
# until it is populated with the final JSON content.

MAX_WAIT_TIME = 10 # Total time to wait for output (in seconds)
sleep_interval = 1 
elapsed_time = 0

while elapsed_time < MAX_WAIT_TIME:
    # Check if the last response part contains content (text or a function response)
    if hasattr(mentor_response[-1], 'content') and mentor_response[-1].content:
        # Check if any text was successfully streamed. If so, break the loop.
        if hasattr(mentor_response[-1].content, 'text') and mentor_response[-1].content.text:
            break
        elif hasattr(mentor_response[-1].content, 'parts') and mentor_response[-1].content.parts:
             if any(p.text for p in mentor_response[-1].content.parts if hasattr(p, 'text')):
                break
    
    # If no final output yet, pause and wait
    # We re-run the runner here to ensure we get a fresh object if the first call was partial/interrupted
    mentor_response = await mentor_runner.run_debug(mentor_prompt) 
    
    await asyncio.sleep(sleep_interval)
    elapsed_time += sleep_interval
    print(f"[Wait: {elapsed_time}s] Waiting for MentorAgent output...")

# Ensure mentor_response is updated after the loop finishes waiting
mentor_response = await mentor_runner.run_debug(mentor_prompt) 


# ---- Extract the FINAL MentorAgent Markdown Output ----
mentor_last = mentor_response[-1]
mentor_text = None

if hasattr(mentor_last, "content") and mentor_last.content is not None:
    if hasattr(mentor_last.content, "text") and mentor_last.content.text:
        mentor_text = mentor_last.content.text
    elif hasattr(mentor_last.content, "parts") and mentor_last.content.parts is not None:
        texts = [p.text for p in mentor_last.content.parts if hasattr(p, "text") and p.text]
        if texts:
            mentor_text = "\n".join(texts)

print("\n===== FINAL MENTOR AGENT OUTPUT =====\n")
print(mentor_text or "[No final text]")

print("\n✅ MentorAgent successfully demonstrated retrieval and updated suggestions.")


 ### Created new session: debug_session_id

User > {
  "student_id": "student_001",
  "progress_update": "\nHi, this is Dev again. Since the last plan:\n- I built a small ML project and uploaded it on GitHub. \n- I am also doing an internship at an AI startup\nWhat should I focus on next?\n"
}


MentorAgent > ## ⭐ Progress Check-in for Dev
---
## Status & Feedback
* **Great Job:** It's fantastic to see you've already built an ML project and uploaded it to GitHub, and you're gaining practical experience through an internship at an AI startup! These are excellent steps that directly align with your goal of specializing in Machine Learning Engineering. Building a portfolio and getting real-world experience are crucial for your development.

## 💡 Updated Suggestions (Next 30-90 Days)
Based on your progress and plan, let's refine your focus:

*   **Master's Program Applications:** As per your medium-term goals, prioritize researching and applying to your shortlisted Master's programs in the Netherlands and Germany. Ensure all application documents are polished and submitted on time.
*   **Portfolio Enhancement:** Continue to showcase your ML project on GitHub. Consider adding a brief technical blog post or README explaining the project's objective, your approach, and the outcomes. 

MentorAgent > ## ⭐ Progress Check-in for Dev
---
## Status & Feedback
* **Great Job:** It's fantastic to see you've already built an ML project and uploaded it to GitHub, and you're gaining practical experience through an internship at an AI startup! These are excellent steps that directly align with your goal of specializing in Machine Learning Engineering. Building a portfolio and getting real-world experience are crucial for your development.

## 💡 Updated Suggestions (Next 30-90 Days)
Based on your progress and plan, let's refine your focus:

*   **Master's Program Applications:** As per your medium-term goals, prioritize researching and applying to your shortlisted Master's programs in the Netherlands and Germany. Ensure all application documents are polished and submitted on time.
*   **Portfolio Enhancement:** Continue to showcase your ML project on GitHub. Consider adding a brief technical blog post or README explaining the project's objective, your approach, and the outcomes. 

## 🚀 Alignment with Capstone Requirements: Ascent AI Submission

This section demonstrates how the **Ascent AI** project directly fulfills all key criteria for the Capstone submission, showcasing advanced multi-agent design, custom tooling, and state management.

***

### 1. Robust Multi-Agent System (MAS)

The system utilizes a structured, sequential MAS for reliable data flow and specialized task execution:

| Agent | Core Function | Data Handoff |
| :--- | :--- | :--- |
| **Profile Agent** | **Data Ingestion & Persistence:** Extracts raw input to build and store a structured **Student Profile**. | Saves profile via **`save_profile_tool`**. |
| **Mapper Agent** | **Strategic Analysis:** Interprets the Profile to propose and select the single **`main_track`** (major). | Passes `main_track` string. |
| **University Agent** | **Constraint Filtering:** Uses the `main_track` to generate the **Reach/Target/Safe university shortlist**. | Passes the structured shortlist. |
| **Action Plan Agent (Implicit)** | **Synthesis & Reporting:** Generates a grade-aware **12–24 month action plan** and produces the final **full pipeline's captured data** in a clear Markdown report. | Saves the final plan via **`save_plan_tool`**. |
| **Mentor Agent (Loop Agent)** | **Continuous Guidance:** Retrieves original plan/profile from memory, updates advice based on student progress, and outputs a **clear Markdown check-in**. | Simulates **multi-session continuity**. |

***

### 2. Custom Tools & Memory Management

The architecture is built on a foundation of structured I/O and persistent storage:

* **Custom Tool – `UniversitySearchTool`:** This key custom tool performs deterministic, reliable filtering of a curated, in-notebook university database against structured student criteria (budget, country, major).
* **Memory Tools:** The suite of four dedicated tools—**`save_profile_tool`, `get_profile_tool`, `save_plan_tool`, `get_plan_tool`**—demonstrates clear separation of concerns for data persistence.

### 3. Sessions & Multi-Session Continuity

* The **Memory Bank** (`STUDENT_MEMORY`, `PLAN_MEMORY`) serves as the central data repository for the student's history.
* The **Mentor Agent** actively utilizes this stored context to simulate **long-running, multi-session guidance** by recalling and updating the student's status over time, fulfilling the requirement for continuous agent interaction.

### 4. Effective Use of Gemini

Gemini models (accessed via the ADK) are strategically deployed for tasks requiring advanced natural language understanding and generative intelligence:

* **Structured Output:** Used for **profile extraction** (converting free-form text to clean JSON).
* **Reasoning:** Used for **creative, justified track mapping** and intelligent **action plan generation**.
* **Mentoring:** Used for **long-term mentoring** and generating **context-aware plan adjustments** in natural language.

In [47]:
# ============================================================
# FINAL OUTPUT SUMMARY AND CREATION OF SUBMISSION FILE FOR KAGGLE(IF NEEDED)
# ============================================================

import json

print("🔄 Running Ascent AI pipeline to regenerate final output...")

demo_description = """
My name is Dev. I recently completed a **Bachelor of Technology (B.Tech) in Computer Science** from an institute in India.
I love Artificial Intelligence (AI), deep learning, data science, and I'm looking for a specialization in Machine Learning. 
I enjoy reading research papers and competing in coding challenges.
My family can afford medium to high tuition, and I'm focused on pursuing a **Master of Science (M.Sc.) in AI in the Netherland* or Germany.
My scores: Overall GPA equivalent to 8.5/10.0 or 3.6/4.0, with top scores in Algorithms and Linear Algebra. I also have one year of experience as a Data Analyst.
"""

prompt = f"""
STUDENT_ID: student_001
STUDENT_DESCRIPTION:
{demo_description}
"""

# Run the pipeline fully again (Profile -> Mapper -> University -> Plan)
demo_response = await runner.run_debug(prompt)

# Extract final turn (ActionPlanAgent)
last_turn = demo_response[-1]
final_text = None

# Safe text extraction (covers text, parts, delta)
if hasattr(last_turn, "content") and last_turn.content:
    if hasattr(last_turn.content, "text") and last_turn.content.text:
        final_text = last_turn.content.text
    elif hasattr(last_turn.content, "parts") and last_turn.content.parts:
        pieces = [p.text for p in last_turn.content.parts if hasattr(p, "text") and p.text]
        if pieces:
            final_text = "\n".join(pieces)

# Fallback
if final_text is None:
    final_text = "[No final text produced by ActionPlanAgent]"

# Try parsing JSON
json_payload = {}
try:
    json_candidate = final_text[final_text.find("{"):]
    json_payload = json.loads(json_candidate)
    print("✅ Parsed final output JSON.")
except Exception as e:
    print("⚠️ JSON parsing failed:", e)
    json_payload = {"raw_output": final_text}

# Save output file
output_filename = "Ascent_output.json"
with open(output_filename, "w") as f:
    json.dump(json_payload, f, indent=2, ensure_ascii=False)

print(f"📁 Saved submission file: {output_filename}")
print("🎉 You can now SUBMIT this file to the competition.")

🔄 Running Clarity AI pipeline to regenerate final output...

 ### Continue session: debug_session_id

User > 
STUDENT_ID: student_001
STUDENT_DESCRIPTION:

My name is Dev. I recently completed a **Bachelor of Technology (B.Tech) in Computer Science** from an institute in India.
I love Artificial Intelligence (AI), deep learning, data science, and I'm looking for a specialization in Machine Learning. 
I enjoy reading research papers and competing in coding challenges.
My family can afford medium to high tuition, and I'm focused on pursuing a **Master of Science (M.Sc.) in AI in the Netherland* or Germany.
My scores: Overall GPA equivalent to 8.5/10.0 or 3.6/4.0, with top scores in Algorithms and Linear Algebra. I also have one year of experience as a Data Analyst.


ProfileAgent > This is a comprehensive plan for your Master's studies in Machine Learning Engineering! It covers all the key aspects from short-term preparation to long-term career goals.

Given this detailed plan, would you

MapperAgent > PHASE 1: Track Mapping
1. Proposed Tracks:
    * Machine Learning Engineering
    * AI Research Scientist
    * Data Science (with AI specialization)

2. Main Track: Machine Learning Engineering

PHASE 2: Action Plan Synthesis & Saving
3. Plan JSON:
```json
{
    "summary": "Pursue a Master of Science in AI/ML in the Netherlands or Germany, specializing in Machine Learning Engineering. Leverage a strong background in Computer Science, AI, deep learning, data science, and excellent scores in Algorithms and Linear Algebra. The goal is to secure a role as a Machine Learning Engineer, with a medium to high tuition budget.",
    "short_term_tasks": [
        "Thoroughly research the specific curricula and admission requirements for AI/ML Master's programs at TU Delft, University of Amsterdam, TU/e, TUM, RWTH Aachen, and Heidelberg University.",
        "Prepare and submit all application documents, including transcripts, CV, letters of recommendation, and English proficiency s